<a href="https://colab.research.google.com/github/diogenesvazmelo/microdados_enem_2018/blob/master/Relat%C3%B3rio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Carrega o Arquivo .csv com os dados

In [0]:
import pandas as pd
path = "/content/drive/My Drive/Estatísticas ENEM/MICRODADOS_ENEM_2018_SAMPLE_43278_TRANSFORMADOS.csv"
dados = pd.read_csv(path)# Dataset is now stored in a Pandas Dataframe

In [0]:
def le_como_array(coluna, caderno):
  from ast import literal_eval
  a = coluna.dropna().apply(literal_eval)
  return (a)

itens_prova_2018 = pd.read_csv("https://raw.githubusercontent.com/diogenesvazmelo/enem-2018/master/ITENS_PROVA_2018.csv")

cadernos = itens_prova_2018['SG_AREA'].unique()

for caderno in cadernos:
  dados['ACERTOS_{}'.format(caderno)] = le_como_array(dados['ACERTOS_{}'.format(caderno)], caderno)
  dados['RESPOSTAS_TRANSFORMADAS_{}'.format(caderno)] = le_como_array(dados['RESPOSTAS_TRANSFORMADAS_{}'.format(caderno)], caderno)
  dados['ACERTOS_TRANSFORMADOS_{}'.format(caderno)] = le_como_array(dados['ACERTOS_TRANSFORMADOS_{}'.format(caderno)], caderno)

## Acertos totais por candidato e caderno 

Acertos de cada categoria (C, CH, MT, LC) - TX_RESPOSTAS e TX_GABARITO

### Scripts

In [0]:
def gerar_acertos_totais_por_candidato_e_caderno(caderno):
  import numpy as np
  itens_prova_2018 = pd.read_csv("https://raw.githubusercontent.com/diogenesvazmelo/enem-2018/master/ITENS_PROVA_2018.csv")
  cadernos = itens_prova_2018['SG_AREA'].unique()
  
  acertos_totais_por_candidato_e_caderno = []
  for linha in range(len(dados['ACERTOS_TRANSFORMADOS_{}'.format(caderno)])):
    acertos_totais_por_candidato_e_caderno.append(np.sum(dados['ACERTOS_TRANSFORMADOS_{}'.format(caderno)][linha]))


  acertos = pd.Series(acertos_totais_por_candidato_e_caderno)
  acertos.index = dados['ACERTOS_TRANSFORMADOS_{}'.format(caderno)].index
  acertos.name = 'ACERTOS_TOTAIS_{}'.format(caderno)
  return acertos

  return(acertos)

In [0]:
acertos = [gerar_acertos_totais_por_candidato_e_caderno('CN'), gerar_acertos_totais_por_candidato_e_caderno('CH'), gerar_acertos_totais_por_candidato_e_caderno('LC'), gerar_acertos_totais_por_candidato_e_caderno('MT')]

In [0]:
dados_com_acertos = dados
dados_com_acertos = dados_com_acertos.join(acertos)
dados = dados_com_acertos
dados_com_acertos = dados_com_acertos[['NU_INSCRICAO', 'NU_NOTA_CN', 'ACERTOS_TOTAIS_CN', 'NU_NOTA_CH', 'ACERTOS_TOTAIS_CH', 'NU_NOTA_LC', 'ACERTOS_TOTAIS_LC', 'NU_NOTA_MT', 'ACERTOS_TOTAIS_MT', 'NU_NOTA_REDACAO']]


In [0]:
dados_com_acertos

In [0]:
def gera_df_nota_media_por_acertos(str_caderno):
  nota_media_por_acertos = dados_com_acertos[['NU_NOTA_{}'.format(str_caderno), 'ACERTOS_TOTAIS_{}'.format(str_caderno)]].dropna().groupby('ACERTOS_TOTAIS_{}'.format(str_caderno)).mean()
  nota_media_por_acertos['desvio_padrao_{}'.format(str_caderno)] = dados_com_acertos[['NU_NOTA_{}'.format(str_caderno), 'ACERTOS_TOTAIS_{}'.format(str_caderno)]].dropna().groupby('ACERTOS_TOTAIS_{}'.format(str_caderno)).std()
  nota_media_por_acertos.rename(columns={'NU_NOTA_{}'.format(str_caderno): 'NU_NOTA_{}_media'.format(str_caderno)}, inplace = True)
  return nota_media_por_acertos

cadernos = ['CN', 'CH', 'LC', 'MT']

nota_media_por_acertos = {}
for caderno in cadernos:
  nota_media_por_acertos[caderno] = gera_df_nota_media_por_acertos(caderno)


### Scripts dos Gráficos

In [0]:
def plota_graficos_errorbar(limite_inferior):
  import seaborn as sns
  import numpy as np
  import matplotlib.pyplot as plt
  import matplotlib.ticker as ticker

  sns.set(style="darkgrid")

  def plotar_grafico_notas_acertos(nota_media_por_acertos, caderno):
    fig = plt.figure(figsize=(20, 10))
    plt.errorbar(nota_media_por_acertos.index, 
                nota_media_por_acertos['NU_NOTA_{}_media'.format(caderno)], 
                nota_media_por_acertos['desvio_padrao_{}'.format(caderno)], 
                linestyle='None', 
                marker='^',
                mfc='red',
                mec='green', ms=10, mew=0.4)

    ax = fig.add_subplot(1, 1, 1)


    # Major ticks every (tanto), minor ticks every (outro tanto)

    start_x, end_x = ax.get_xlim()

    major_ticks_x = np.arange(start_x-(start_x%10)+10, 
                              nota_media_por_acertos.index.max()+2, 
                              1)
    minor_ticks_x = np.arange(start_x-(start_x%10)+10, 
                              end_x-(end_x%10)+10, 
                              1)


    start_y, end_y = ax.get_ylim()

    major_ticks_y = np.arange(start_y-(start_y%10)+10, 
                              end_y, 
                              10)
    minor_ticks_y = np.arange(start_y-(start_y%10)+10, 
                              end_y,
                              5)

    ax.set_xticks(major_ticks_x)
    #ax.set_xticks(minor_ticks_x, minor=True)
    ax.set_yticks(major_ticks_y)
    ax.set_yticks(minor_ticks_y, minor=True)

    # And a corresponding grid
    ax.grid(which='both')

    # Or if you want different settings for the grids:
    ax.grid(which='minor', alpha=10.0)
    ax.grid(which='major', alpha=10.0)

    plt.title('Nota média por acertos no caderno de {}'.format(caderno))
    ax.set_ylabel('Notas médias')
    ax.set_xlabel('Quantidades de acertos')

    plt.show()

  cadernos = ['CN', 'CH', 'LC', 'MT']
  for caderno in cadernos:
    nota_media_por_acertos_acima_de_20 = nota_media_por_acertos[caderno].query('index >= {}'.format(limite_inferior))
    plotar_grafico_notas_acertos(nota_media_por_acertos_acima_de_20, caderno)

In [0]:
def plota_graficos_errorbar_2(limite_inferior, limite_superior):
  import seaborn as sns
  import numpy as np
  import matplotlib.pyplot as plt
  import matplotlib.ticker as ticker

  sns.set(style="darkgrid")

  def plotar_grafico_notas_acertos(nota_media_por_acertos, caderno):
    fig = plt.figure(figsize=(20, 10))
    plt.errorbar(nota_media_por_acertos.index, 
                nota_media_por_acertos['NU_NOTA_{}_media'.format(caderno)], 
                nota_media_por_acertos['desvio_padrao_{}'.format(caderno)], 
                linestyle='None', 
                marker='^',
                mfc='red',
                mec='green', ms=10, mew=0.4)

    ax = fig.add_subplot(1, 1, 1)


    # Major ticks every (tanto), minor ticks every (outro tanto)

    start_x, end_x = ax.get_xlim()

    major_ticks_x = np.arange(start_x-(start_x%10)+0, 
                              nota_media_por_acertos.index.max()+2, 
                              1)
    minor_ticks_x = np.arange(start_x-(start_x%10)+10, 
                              end_x-(end_x%10)+10, 
                              1)


    start_y, end_y = ax.get_ylim()

    major_ticks_y = np.arange(start_y-(start_y%10)+10, 
                              end_y, 
                              10)
    minor_ticks_y = np.arange(start_y-(start_y%10)+10, 
                              end_y,
                              5)

    ax.set_xticks(major_ticks_x)
    #ax.set_xticks(minor_ticks_x, minor=True)
    ax.set_yticks(major_ticks_y)
    ax.set_yticks(minor_ticks_y, minor=True)

    # And a corresponding grid
    ax.grid(which='both')

    # Or if you want different settings for the grids:
    ax.grid(which='minor', alpha=10.0)
    ax.grid(which='major', alpha=10.0)

    plt.title('Nota média por acertos no caderno de {}'.format(caderno))
    ax.set_ylabel('Notas médias')
    ax.set_xlabel('Quantidades de acertos')

    plt.show()

  cadernos = ['CN', 'CH', 'LC', 'MT']
  for caderno in cadernos:
    nota_media_por_acertos_acima_de_20 = nota_media_por_acertos[caderno].query('index >= {} and index <= {}'.format(limite_inferior, limite_superior))
    plotar_grafico_notas_acertos(nota_media_por_acertos_acima_de_20, caderno)

### Tabelas

In [0]:
nota_media_por_acertos['CN']

In [0]:
nota_media_por_acertos['CH']

In [0]:
nota_media_por_acertos['LC']

In [0]:
nota_media_por_acertos['MT']

### Gráficos Errorbar



In [0]:
plota_graficos_errorbar(20)

In [0]:
plota_graficos_errorbar_2(1, 19)

### Gráficos Boxplot

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

sns.set(style="darkgrid")

def plotar_grafico_notas_acertos(nota_media_por_acertos, caderno):
  fig = plt.figure(1, figsize=(20, 10))
  sns.boxplot(x=nota_media_por_acertos['ACERTOS_TOTAIS_{}'.format(caderno)], 
              y=nota_media_por_acertos['NU_NOTA_{}'.format(caderno)],
              data=nota_media_por_acertos)

  ax = fig.add_subplot(111)

  start_x, end_x = ax.get_xlim()
  ax.set_xticks(np.arange(start_x-(start_x%10)+10, 
                          end_x, 
                          1))

  start_y, end_y = ax.get_ylim()
  ax.set_yticks(np.arange(start_y-(start_y%10)+10, 
                          end_y, 
                          10))
  #ax.xaxis.set_major_locator(ticker.MultipleLocator())
  ax.xaxis.set_major_formatter(ticker.ScalarFormatter(-(nota_media_por_acertos['ACERTOS_TOTAIS_{}'.format(caderno)].min())))
  
  ax.grid(True)


  plt.title('Boxplot notas por acertos no caderno de {}'.format(caderno))
  ax.set_ylabel('Nota')
  ax.set_xlabel('Quantidades de acertos')

  plt.show()

for caderno in cadernos:
  notas_por_acertos_acima_de_20 = dados_com_acertos[['NU_NOTA_{}'.format(caderno), 'ACERTOS_TOTAIS_{}'.format(caderno)]].dropna().query('ACERTOS_TOTAIS_{} >= 20'.format(caderno))
  plotar_grafico_notas_acertos(notas_por_acertos_acima_de_20, caderno)


### Coisas para se explorar

In [0]:
#### Correlação acertos (acima de 20) com o desvio padrão das notas

dados_com_acertos_e_std = dados_com_acertos

for caderno in cadernos:
  dados_com_acertos_e_std = dados_com_acertos_e_std.join(nota_media_por_acertos['{}'.format(caderno)]['desvio_padrao_{}'.format(caderno)], on='ACERTOS_TOTAIS_{}'.format(caderno))

dados_com_acertos_e_std = dados_com_acertos_e_std[['NU_NOTA_CN', 'ACERTOS_TOTAIS_CN', 'desvio_padrao_CN','NU_NOTA_CH', 'ACERTOS_TOTAIS_CH', 'desvio_padrao_CH', 'NU_NOTA_LC', 'ACERTOS_TOTAIS_LC', 'desvio_padrao_LC', 'NU_NOTA_MT', 'ACERTOS_TOTAIS_MT', 'desvio_padrao_MT','NU_NOTA_REDACAO']]
dados_com_acertos_e_std

for caderno in cadernos:
  result = nota_media_por_acertos['{}'.format(caderno)].reset_index()[['ACERTOS_TOTAIS_{}'.format(caderno), 'desvio_padrao_{}'.format(caderno)]].query('ACERTOS_TOTAIS_{} >= 20'.format(caderno)).corr()
  print(result)


## Quantidade de acertos por questão - Termos absolutos

### Script

In [0]:
def gerar_num_acertos_por_questao(LISTAS_ACERTOS, caderno):
  #if (caderno == 'LC'):
  quantidade_acertos = [0] * 50
  #else:
  #  quantidade_acertos = [0] * 45

  for i in range(len(l)):
    for j in range(len(l[i])):
      #print(LISTAS_ACERTOS[i][j])
      if (l[i][j] == 1):
        quantidade_acertos[j] += 1

  return quantidade_acertos


total_acertos_por_questao_dict = {}

cadernos = ['CN', 'CH', 'LC', 'MT']

for caderno in cadernos:
  l = dados['ACERTOS_TRANSFORMADOS_{}'.format(caderno)].dropna().to_list()
  total_acertos = gerar_num_acertos_por_questao(l, caderno)

  total_acertos = pd.Series(total_acertos)
  total_acertos_por_questao_dict[caderno] = total_acertos

total_acertos_por_questao = pd.DataFrame(total_acertos_por_questao_dict)





total_acertos_por_questao.index.name = 'Questao'
total_acertos_por_questao = total_acertos_por_questao.reset_index()

import pandas as pd
um_ate_50 = pd.Series(range(1,51))
total_acertos_por_questao['Questao'] = um_ate_50


In [0]:
total_acertos_por_questao_nan = total_acertos_por_questao.replace({0:np.nan})
# A linha abaixo substitui tanto zeros int quanto zeros string para nan
# total_acertos_por_questao_nan = total_acertos_por_questao.replace({'0':np.nan, 0:np.nan}) 

### Tabelas

In [0]:
total_acertos_por_questao_nan

In [0]:
total_acertos_por_questao

In [0]:
total_acertos_por_questao.sort_values(by='MT', ascending=False)

### Scripts dos Gráficos

In [0]:
def plotar_grafico_acertos_por_questao_relplot(caderno):
  import seaborn as sns
  sns.relplot(x="index", 
              y="{}".format(caderno), 
              #hue="origin", 
              size="{}".format(caderno),
              sizes=(40, 400), alpha=.9, palette="muted",
              height=6, 
              data=total_acertos_por_questao.reset_index().query('{} > 0'.format(caderno)))

In [0]:
##### Esses imports são para trabalhar localmente 

# import pandas as pd
# import plotly.offline as py
# import plotly.graph_objs as go
# py.init_notebook_mode(connected=True)

In [0]:
import pandas as pd
import numpy as np

from scipy import stats 
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go

#py.init_notebook_mode(connected=False)

In [0]:
# Devido o Plotly estar sendo utilizado no Google Collab, precisaremos definir
# a função abaixo e chamá-la sempre que quisermos exibir um gráfico

def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

In [0]:
import plotly.offline as py

configure_plotly_browser_state()
py.init_notebook_mode(connected=False)

data = []


def cria_grafico_scatter_acertos(caderno):
  trace = go.Scatter(x = total_acertos_por_questao['Questao'],
                    y = total_acertos_por_questao['{}'.format(caderno)],
                    name = '{}'.format(caderno),
                    mode = 'markers')
  return trace

cadernos = ['CN', 'CH', 'LC', 'MT']
for caderno in cadernos:
  data.append(cria_grafico_scatter_acertos(caderno))

# data = [trace]

py.iplot(data)

In [0]:
import plotly.offline as py

configure_plotly_browser_state()
py.init_notebook_mode(connected=False)

data = []


def cria_grafico_scatter_acertos(caderno):
  trace = go.Scatter(x = total_acertos_por_questao_nan['Questao'],
                    y = total_acertos_por_questao_nan['{}'.format(caderno)],
                    name = '{}'.format(caderno),
                    mode = 'markers')
  return trace

cadernos = ['CN', 'CH', 'LC', 'MT']
for caderno in cadernos:
  data.append(cria_grafico_scatter_acertos(caderno))

# data = [trace]
layout = go.Layout(title = 'Dispersão de acertos para cada caderno do ENEM 2018',
                   titlefont = {'family': 'Arial',
                                'size': 22,
                                'color': '#7f7f7f'},
                   xaxis = {'title': 'Número da questão (com base na prova de cor amarela)'},
                   yaxis = {'title': 'Quantidade de Acertos'},
                   #paper_bgcolor = 'rgb(243, 243, 243)',
                   #plot_bgcolor = 'rgb(243, 243, 243)'
                   )


fig = go.Figure(data=data, layout=layout)
fig.update_xaxes(tick0=0, dtick=1)
fig.update_xaxes(range=[0.5, 50.5])
fig.update_yaxes(tick0=10000, dtick=5000)
py.iplot(fig)

In [0]:
def gera_acertos_por_questao_plotly(titulo, label_eixo_x, label_eixo_y):
  import plotly.offline as py

  configure_plotly_browser_state()
  py.init_notebook_mode(connected=False)

  data = []


  def cria_grafico_scatter_acertos(caderno):
    trace = go.Scatter(x = total_acertos_por_questao_nan['Questao'],
                      y = total_acertos_por_questao_nan['{}'.format(caderno)],
                      name = '{}'.format(caderno),
                      mode = 'markers')
    return trace

  cadernos = ['CN', 'CH', 'LC', 'MT']
  for caderno in cadernos:
    data.append(cria_grafico_scatter_acertos(caderno))

  # data = [trace]
  layout = go.Layout(title = '{}'.format(titulo),
                    titlefont = {'family': 'Arial',
                                  'size': 22,
                                  'color': '#7f7f7f'},
                    xaxis = {'title': '{}'.format(label_eixo_x)},
                    yaxis = {'title': '{}'.format(label_eixo_y)},
                    #paper_bgcolor = 'rgb(243, 243, 243)',
                    #plot_bgcolor = 'rgb(243, 243, 243)'
                    )


  fig = go.Figure(data=data, layout=layout)
  fig.update_xaxes(tick0=0, dtick=1)
  fig.update_xaxes(range=[0.5, 50.5])
  fig.update_yaxes(tick0=10000, dtick=5000)
  py.iplot(fig)

In [0]:
total_acertos_por_questao.index.values

### Gráficos

In [0]:
titulo = 'Total de acertos por questão do ENEM 2018'
label_eixo_x = 'Número da questão (com base na prova de cor amarela)'
label_eixo_y = 'Quantidade de Acertos'

gera_acertos_por_questao_plotly(titulo, label_eixo_x, label_eixo_y)

In [0]:
#plotar_grafico_acertos_por_questao_relplot('CN')

## Quantidade de acertos por questão - Porcentagem

### Script

In [0]:
total_acertos_porcentagem = total_acertos_por_questao_nan

cadernos = ['CN', 'CH', 'LC', 'MT']
for caderno in cadernos:
  quantidade = len(dados[['ACERTOS_TOTAIS_{}'.format(caderno)]].dropna())
  total_acertos_porcentagem['PORCENTAGEM_DE_ACERTOS_{}'.format(caderno)] = total_acertos_por_questao_nan['{}'.format(caderno)]*100/quantidade

### Tabelas

In [0]:
total_acertos_porcentagem

### Script dos Gráficos

In [0]:
def gera_porcentagem_acertos_por_questao_plotly(titulo, label_eixo_x, label_eixo_y):
  import plotly.offline as py

  configure_plotly_browser_state()
  py.init_notebook_mode(connected=False)

  data = []


  def cria_grafico_scatter_acertos(caderno):
    trace = go.Scatter(x = total_acertos_porcentagem['Questao'],
                      y = total_acertos_porcentagem['PORCENTAGEM_DE_ACERTOS_{}'.format(caderno)],
                      name = '{}'.format(caderno),
                      mode = 'markers')
    return trace

  cadernos = ['CN', 'CH', 'LC', 'MT']
  for caderno in cadernos:
    data.append(cria_grafico_scatter_acertos(caderno))

  # data = [trace]
  layout = go.Layout(title = '{}'.format(titulo),
                    titlefont = {'family': 'Arial',
                                  'size': 22,
                                  'color': '#7f7f7f'},
                    xaxis = {'title': '{}'.format(label_eixo_x)},
                    yaxis = {'title': '{}'.format(label_eixo_y)},
                    #paper_bgcolor = 'rgb(243, 243, 243)',
                    #plot_bgcolor = 'rgb(243, 243, 243)'
                    )


  fig = go.Figure(data=data, layout=layout)
  fig.update_xaxes(tick0=0, dtick=1)
  fig.update_xaxes(range=[0.5, 50.5])
  fig.update_yaxes(tick0=0, dtick=5)
  py.iplot(fig)

### Gráficos

In [0]:
titulo = 'Porcentagem de acertos para cada questão do ENEM 2018'
label_eixo_x = 'Número da questão (com base na prova de cor amarela)'
label_eixo_y = 'Porcentagem de Acertos'

gera_porcentagem_acertos_por_questao_plotly(titulo, label_eixo_x, label_eixo_y)

### Questões com mais erros e mais acertos

**CN**

> Mais acertos





In [0]:
caderno = 'CN'
total_acertos_porcentagem[['Questao', 'PORCENTAGEM_DE_ACERTOS_{}'.format(caderno)]].sort_values(by='PORCENTAGEM_DE_ACERTOS_{}'.format(caderno), ascending=False).head(15)

> Mais Erros







In [0]:
caderno = 'CN'
total_acertos_porcentagem[['Questao', 'PORCENTAGEM_DE_ACERTOS_{}'.format(caderno)]].sort_values(by='PORCENTAGEM_DE_ACERTOS_{}'.format(caderno), ascending=True).head()

**CH**



> Mais acertos





In [0]:
caderno = 'CH'
total_acertos_porcentagem[['Questao', 'PORCENTAGEM_DE_ACERTOS_{}'.format(caderno)]].sort_values(by='PORCENTAGEM_DE_ACERTOS_{}'.format(caderno), ascending=False).head()

> Mais Erros





In [0]:
caderno = 'CH'
total_acertos_porcentagem[['Questao', 'PORCENTAGEM_DE_ACERTOS_{}'.format(caderno)]].sort_values(by='PORCENTAGEM_DE_ACERTOS_{}'.format(caderno), ascending=True).head()

**LC**



> Mais acertos





In [0]:
caderno = 'LC'
total_acertos_porcentagem[['Questao', 'PORCENTAGEM_DE_ACERTOS_{}'.format(caderno)]].sort_values(by='PORCENTAGEM_DE_ACERTOS_{}'.format(caderno), ascending=False).head()

> Mais Erros





In [0]:
caderno = 'LC'
total_acertos_porcentagem[['Questao', 'PORCENTAGEM_DE_ACERTOS_{}'.format(caderno)]].sort_values(by='PORCENTAGEM_DE_ACERTOS_{}'.format(caderno), ascending=True).head()

**MT**



> Mais acertos





In [0]:
caderno = 'MT'
total_acertos_porcentagem[['Questao', 'PORCENTAGEM_DE_ACERTOS_{}'.format(caderno)]].sort_values(by='PORCENTAGEM_DE_ACERTOS_{}'.format(caderno), ascending=False).head()

> Mais Erros





In [0]:
caderno = 'MT'
total_acertos_porcentagem[['Questao', 'PORCENTAGEM_DE_ACERTOS_{}'.format(caderno)]].sort_values(by='PORCENTAGEM_DE_ACERTOS_{}'.format(caderno), ascending=True).head()

## Média de acertos

In [0]:
dados_com_acertos

In [0]:
def gera_media_acertos_por_caderno_plotly(titulo, label_eixo_x, label_eixo_y, coluna):
  import plotly.offline as py

  configure_plotly_browser_state()
  py.init_notebook_mode(connected=False)

  data = []


  def cria_grafico_boxplot_acertos(caderno, coluna):
    trace = go.Box(#x = total_acertos_porcentagem['Questao'],
                      y = dados_com_acertos['{}_{}'.format(coluna, caderno)],
                      name = '{}'.format(caderno))
    return trace

  cadernos = ['CN', 'CH', 'LC', 'MT']
  for caderno in cadernos:
    data.append(cria_grafico_boxplot_acertos(caderno, coluna))

  # data = [trace]
  layout = go.Layout(title = '{}'.format(titulo),
                    titlefont = {'family': 'Arial',
                                  'size': 22,
                                  'color': '#7f7f7f'},
                    xaxis = {'title': '{}'.format(label_eixo_x)},
                    yaxis = {'title': '{}'.format(label_eixo_y)},
                    #paper_bgcolor = 'rgb(243, 243, 243)',
                    #plot_bgcolor = 'rgb(243, 243, 243)'
                    )


  fig = go.Figure(data=data, layout=layout)
  # fig.update_xaxes(tick0=0, dtick=1)
  # fig.update_xaxes(range=[0.5, 50.5])
  # fig.update_yaxes(tick0=0, dtick=5)
  py.iplot(fig)

In [0]:
gera_media_acertos_por_caderno_plotly('a', 'b', 'c', 'NU_NOTA')

In [0]:
gera_media_acertos_por_caderno_plotly('a', 'b', 'c', 'ACERTOS_TOTAIS')

## Outros

In [0]:
def gera_coluna_qual_e_a_questao_acertou_uma(caderno, index, tabela):
  coluna = []
  for linha in tabela['ACERTOS_TRANSFORMADOS_{}'.format(caderno)].to_list():
    for i in range(len(linha)):
      if (linha[i] == 1):
        coluna.append(i+1)
  
  import pandas as pd
  coluna = pd.Series(coluna)
  coluna.name = 'qual_e_a_unica_questao_que_acertou_{}'.format(caderno)
  coluna.index = index
  return (coluna)

In [0]:
def gerar_tabela_quem_acertou_so_uma(caderno):
  tabela = dados_com_acertos.query('ACERTOS_TOTAIS_{} == 1'.format(caderno))[['NU_INSCRICAO', 'NU_NOTA_{}'.format(caderno)]]
  tabela = tabela.join(dados[['ACERTOS_TRANSFORMADOS_{}'.format(caderno)]])
  pd.set_option('display.max_colwidth', 200)
  #pd.options.display.width = 0

  return tabela.join(gera_coluna_qual_e_a_questao_acertou_uma(caderno, tabela.index, tabela))

### Pessoal que acertou apenas 1 questão

In [0]:
caderno = 'CN'
gerar_tabela_quem_acertou_so_uma(caderno).sort_values(by='NU_NOTA_{}'.format(caderno), ascending=False)

In [0]:
caderno = 'CH'
gerar_tabela_quem_acertou_so_uma(caderno).sort_values(by='NU_NOTA_{}'.format(caderno), ascending=False)

In [0]:
caderno = 'LC'
gerar_tabela_quem_acertou_so_uma(caderno).sort_values(by='NU_NOTA_{}'.format(caderno), ascending=False)

In [0]:
caderno = 'MT'
gerar_tabela_quem_acertou_so_uma(caderno).sort_values(by='NU_NOTA_{}'.format(caderno), ascending=False)

In [0]:
gerar_tabela_quem_acertou_so_uma(caderno).sort_values(by='NU_NOTA_{}'.format(caderno), ascending=False)['qual_e_a_unica_questao_que_acertou_MT'].unique()

### Pessoal que acertou 2 questões



In [0]:
def verifica_se_acertou_a_questao_tal(caderno, index, tabela):
  coluna = []
  for linha in tabela['ACERTOS_TRANSFORMADOS_{}'.format(caderno)].to_list():
    if (linha[16] == 1):
        coluna.append(16+1)
    else:
        coluna.append(0)
  
  import pandas as pd
  coluna = pd.Series(coluna)
  coluna.name = 'se_acertou_a_questao_tal_{}'.format(caderno)
  coluna.index = index
  return (coluna)

In [0]:
def gera_coluna_qual_e_a_segunda_questao_que_acertou(caderno, index, tabela):
  coluna = []
  for linha in tabela['ACERTOS_TRANSFORMADOS_{}'.format(caderno)].to_list():
    for i in range(len(linha)):
      if (linha[i] == 1):
        coluna.append(i+1)
  
  import pandas as pd
  coluna = pd.Series(coluna)
  coluna.name = 'qual_e_a_segunda_questao_que_acertou_{}'.format(caderno)
  coluna.index = index
  return (coluna)

In [0]:
def gerar_tabela_quem_acertou_duas(caderno):
  tabela = dados_com_acertos.query('ACERTOS_TOTAIS_{} == 2'.format(caderno))[['NU_INSCRICAO', 'NU_NOTA_{}'.format(caderno)]]
  tabela = tabela.join(dados[['ACERTOS_TRANSFORMADOS_{}'.format(caderno)]])
  
  tabela = tabela.join(verifica_se_acertou_a_questao_tal(caderno, tabela.index, tabela))

  tabela = tabela.query('se_acertou_a_questao_tal_{} == 17'.format(caderno))
  pd.set_option('display.max_colwidth', 200)
  #pd.options.display.width = 0

  return tabela

 # return tabela.join(gera_coluna_qual_e_a_questao_acertou_uma(caderno, tabela.index, tabela))

In [0]:
caderno = 'CN'
gerar_tabela_quem_acertou_duas(caderno).sort_values(by='NU_NOTA_{}'.format(caderno), ascending=False)

In [0]:
caderno = 'CH'
gerar_tabela_quem_acertou_so_uma(caderno).sort_values(by='NU_NOTA_{}'.format(caderno), ascending=False)

In [0]:
caderno = 'LC'
gerar_tabela_quem_acertou_so_uma(caderno).sort_values(by='NU_NOTA_{}'.format(caderno), ascending=False)

In [0]:
caderno = 'MT'
gerar_tabela_quem_acertou_so_uma(caderno).sort_values(by='NU_NOTA_{}'.format(caderno), ascending=False)

In [0]:
gerar_tabela_quem_acertou_so_uma(caderno).sort_values(by='NU_NOTA_{}'.format(caderno), ascending=False)['qual_e_a_unica_questao_que_acertou_MT'].unique()

### Acertou 44

In [0]:
def gerar_tabela_quem_acertou_duas(caderno):
  tabela = dados_com_acertos.query('ACERTOS_TOTAIS_{} == 44'.format(caderno))[['NU_INSCRICAO', 'NU_NOTA_{}'.format(caderno)]]
  tabela = tabela.join(dados[['ACERTOS_TRANSFORMADOS_{}'.format(caderno)]])
  
  tabela = tabela.join(verifica_se_acertou_a_questao_tal(caderno, tabela.index, tabela))

  tabela = tabela.query('se_acertou_a_questao_tal_{} == 17'.format(caderno))
  pd.set_option('display.max_colwidth', 200)
  #pd.options.display.width = 0

  return tabela

 # return tabela.join(gera_coluna_qual_e_a_questao_acertou_uma(caderno, tabela.index, tabela))

In [0]:
caderno = 'CN'
gerar_tabela_quem_acertou_duas(caderno).sort_values(by='NU_NOTA_{}'.format(caderno), ascending=False)

# New Section

In [0]:
def verifica_se_acertou_a_questao_tal(caderno, tabela, coluna):
  for linha in tabela['ACERTOS_TRANSFORMADOS_{}'.format(caderno)].to_list():
    for questao in range(len(linha)):
      if (linha[questao] == 1):
          coluna[questao] = True
  
  return (coluna)

In [0]:
caderno = 'MT'
num_de_questoes_no_caderno = 44


for i in range(45):
  coluna = []
  coluna = coluna + [False] * 45
  tabela = dados_com_acertos.query('ACERTOS_TOTAIS_{} == {}'.format(caderno, i))[['NU_INSCRICAO', 'NU_NOTA_{}'.format(caderno)]]
  tabela = tabela.join(dados[['ACERTOS_TRANSFORMADOS_{}'.format(caderno)]])
  coluna = verifica_se_acertou_a_questao_tal(caderno, tabela, coluna)
  
  contador = 0
  for j in coluna:
    if j == True:
      contador += 1
    if contador == num_de_questoes_no_caderno:
      print(i)
      break

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43


In [0]:
def questoes_que_acertou(caderno, quantidade_de_questoes, tabela, index):
  lista_questoes_acertadas = []
  #for qt_questoes in range(quantidade_de_questoes):
  for linha in tabela['ACERTOS_TRANSFORMADOS_{}'.format(caderno)].to_list():
    questoes_acertadas = []
    for quest in range(len(linha)):
      if linha[quest] == 1:
        questoes_acertadas.append(quest + 1)
    lista_questoes_acertadas.append(questoes_acertadas)
  
  import pandas as pd
  lista_questoes_acertadas = pd.Series(lista_questoes_acertadas)
  lista_questoes_acertadas.name = 'quais_acertou_{}'.format(caderno)
  lista_questoes_acertadas.index = index

  return (lista_questoes_acertadas)

In [0]:
caderno ='MT'
quantidade_de_acertos = 1


tabela = dados_com_acertos.query('ACERTOS_TOTAIS_{} == {}'.format(caderno, quantidade_de_acertos))[['NU_INSCRICAO', 'NU_NOTA_{}'.format(caderno)]]
tabela = tabela.join(dados[['ACERTOS_TRANSFORMADOS_{}'.format(caderno)]])

tabela = tabela.join(questoes_que_acertou(caderno, quantidade_de_acertos, tabela, tabela.index))

import pandas as pd
pd.set_option('display.max_rows', None)

dificuldade = tabela.sort_values('NU_NOTA_{}'.format(caderno), ascending=False)['quais_acertou_MT']

tabela.sort_values('NU_NOTA_{}'.format(caderno), ascending=False)


,NU_INSCRICAO,NU_NOTA_MT,ACERTOS_TRANSFORMADOS_MT,quais_acertou_MT
66600,180007765836,407.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[11]
23739,180011593833,407.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[11]
81131,180010705384,407.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[11]
109748,180012106509,401.7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[21]
118113,180013368496,386.4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[25]
87594,180011922251,386.2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[35]
29215,180008226509,378.9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[23]
77140,180011361668,378.9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[23]
39476,180009129992,374.8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]",[40]
80782,180011191391,373.7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[14]


In [0]:
dif = []
for linha in dificuldade:
  for quest in range(len(linha)):
    if linha[quest] not in dif:
      dif.append(linha[quest])

dificuldade = dif
dif = 0
dificuldade

TypeError: ignored

In [0]:
dificuldade

[11, 21, 25, 35, 23, 40, 14, 36, 8, 29, 7, 32, 22, 1, 28, 31, 43, 42, 37]

In [0]:
len(dificuldade)

19

In [0]:
caderno ='MT'
quantidade_de_acertos = 3


tabela = dados_com_acertos.query('ACERTOS_TOTAIS_{} == {}'.format(caderno, quantidade_de_acertos))[['NU_INSCRICAO', 'NU_NOTA_{}'.format(caderno)]]
tabela = tabela.join(dados[['ACERTOS_TRANSFORMADOS_{}'.format(caderno)]])

tabela = tabela.join(questoes_que_acertou(caderno, quantidade_de_acertos, tabela, tabela.index))

import pandas as pd
pd.set_option('display.max_rows', 10)

tabela.sort_values('NU_NOTA_{}'.format(caderno), ascending=True).head(10)

,NU_INSCRICAO,NU_NOTA_MT,ACERTOS_TRANSFORMADOS_MT,quais_acertou_MT
85081,180008212260,370.1,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 24, 26]"
42260,180010964907,370.2,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[9, 42, 43]"
92097,180011116047,370.5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[16, 27, 37]"
91382,180013411959,370.5,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[9, 16, 42]"
61467,180010144515,370.6,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[9, 33, 42]"
48669,180008372088,370.9,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 16, 39]"
89081,180012694664,370.9,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[9, 31, 37]"
33755,180007845083,371.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[13, 37, 42]"
120394,180010311183,371.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[10, 17, 43]"
34848,180010001979,371.1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[9, 28, 43]"


In [0]:
caderno ='MT'
quantidade_de_acertos = 2


tabela = dados_com_acertos.query('ACERTOS_TOTAIS_{} == {}'.format(caderno, quantidade_de_acertos))[['NU_INSCRICAO', 'NU_NOTA_{}'.format(caderno)]]
tabela = tabela.join(dados[['ACERTOS_TRANSFORMADOS_{}'.format(caderno)]])

tabela = tabela.join(questoes_que_acertou(caderno, quantidade_de_acertos, tabela, tabela.index))

import pandas as pd
pd.set_option('display.max_rows', 10)


a = tabela.sort_values('NU_NOTA_{}'.format(caderno), ascending=False)['quais_acertou_MT']

for i in range(46):
  for j in a:
    # print(j)  
    if i in j and i in dificuldade:
      print('-->' + str(i))

-->1
-->1
-->1
-->7
-->7
-->8
-->8
-->8
-->8
-->8
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->11
-->14
-->14
-->14
-->14
-->14
-->14
-->21
-->21
-->21
-->21
-->21
-->21
-->21
-->21
-->22
-->22
-->22
-->22
-->22
-->23
-->23
-->23
-->23
-->25
-->25
-->25
-->25
-->25
-->25
-->25
-->25
-->28
-->28
-->28
-->28
-->29
-->29
-->29
-->29
-->29
-->29
-->29
-->29
-->29
-->29
-->31
-->31
-->32
-->32
-->32
-->32
-->32
-->32
-->32
-->35
-->36
-->36
-->36
-->36
-->36
-->36
-->37
-->37
-->37
-->37
-->37
-->40
-->40
-->40
-->40
-->40
-->40
-->40
-->40
-->40
-->40
-->42
-->42
-->42
-->42
-->42
-->42
-->42
-->43
-->43
-->43
-->43
-->43


In [0]:
dificuldade = dificuldade.to_list()
dificuldade


[[11],
 [11],
 [11],
 [21],
 [25],
 [35],
 [23],
 [23],
 [40],
 [14],
 [14],
 [36],
 [8],
 [8],
 [29],
 [7],
 [32],
 [32],
 [22],
 [1],
 [28],
 [31],
 [43],
 [42],
 [37]]

In [0]:
for i in range(46): 
  if i not in dificuldade and i > 0:
    for i 


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45


# Topological Sort

In [0]:
 def alienOrder(self, words):  
           if not len(words): return ''  
           order = []  
           for pair in list(zip(words, words[1:])):  
                for x, y in zip(*pair):  
                     if x != y:  
                          order.append(x + y)  
                          break  
           letters = set(''.join(words))  
           dictionary = []  
           while order:
                #print(list(zip(*order))[1])
                #teste = list(zip(*order))[1]
                teste = list(zip(*order))
                if len(teste) > 0:
                  teste = teste[1]
                  print(teste)
                  print('----------')
                  free_letter = letters - (set(teste))
                else:
                  return ''.join(dictionary + list(letters))  
                if not free_letter: 
                  return ''  

                dictionary += free_letter  
                order = filter(free_letter.isdisjoint, order)  
                letters -= free_letter  
           return ''.join(dictionary + list(letters))  

In [0]:
palavras = ["wrt","wrf","er","ett","rftt"]
#palavras = 'oi'
alienOrder(3, palavras)

('f', 'e', 't', 'r')
----------
('f', 't', 'r')
----------


'wetrf'

In [0]:
#  def alienOrder(self, words):  
#            if not len(words): return ''  
#            order=[]  
#            for pair in zip(words,words[1:]):  
#                 for x,y in zip(*pair):  
#                      if x!=y:  
#                           order.append(x+y)  
#                           break  
#            letters=set(''.join(words))  
#            dictionary=[]  
#            while order:  
#                 free_letter=letters-set(zip(*order)[1])  
#                 if not free_letter: return ''  
#                 dictionary+=free_letter  
#                 order=filter(free_letter.isdisjoint,order)  
#                 letters-=free_letter  
#            return ''.join(dictionary+list(letters)) 